# Extracción masiva de datos alojados en Cassandra - Precipitación

> Elaborado por Paola Álvarez, profesional contratista IDEAM, contrato 196 de 2024. Comentarios o inquietudes, remitir a palvarez@ideam.gov.co

**Se importan librerías**

In [ ]:
import pandas as pd
import datetime
import prestodb
import os
from datetime import datetime

In [ ]:
conn=prestodb.dbapi.connect(
    host='172.16.50.20',
    port=8080,
    user='Paola',
    catalog='raw',
    schema='cassandra',
)
cur = conn.cursor()

_________________

## Pruebas

In [ ]:
# Extracción corta: estaciones y sensores específicos
cur.execute("select station, sensor, event_time + interval '5' hour event_time, event_value from cassandra.raw.last_observations where station in ('0011045010','2403000114','2403000113','0021206960','2621500070','0016015130') and sensor in ('0240','0407')")
ultimos_datos_precip = cur.fetchall()

In [ ]:
inicio = '2003-07-01 00:00:00.000'
fin =    '2023-08-31 23:59:00.000'

In [ ]:
### Pruebas seleccionando parámetro general: falla por el alcance del servidor, se considera reducir fechas
cur.execute(f''' select lm.station, s.name, lm.sensor, lm.event_time + interval '5' hour event_time, lm.event_value 
from cassandra.raw.last_month_observations AS lm INNER JOIN cassandra.raw.stations AS s
    ON lm.station = s.stationid
where lm.event_time BETWEEN timestamp '{inicio}' AND timestamp '{fin}' 
AND lm.sensor in ('0240','0319','0249', '0311', 'PTPM_CON') ''')
ultimas_obs_dia = cur.fetchall()

In [ ]:
inicio = '2003-07-01 00:00:00.000'
fin =    '2023-08-31 23:59:00.000'

In [ ]:
#### Pruebas seleccionando una sola estación y sensor específico de precipitación diezminutal
# Extracción de datos de estaciones determinadas
cur.execute(f''' select lm.station, s.name, lm.sensor, lm.event_time + interval '5' hour event_time, lm.event_value 
from cassandra.raw.weather_events AS lm INNER JOIN cassandra.raw.stations AS s
    ON lm.station = s.stationid
where lm.station in ('2120000108')  
AND lm.event_time BETWEEN timestamp '{inicio}' AND timestamp '{fin}' 
AND lm.sensor in ('0240') ''')
Villeta = cur.fetchall()

In [ ]:
# Se genera un Data Frame a partir de los datos extaídos
Villeta0240 = pd.DataFrame(Villeta, columns= ['Station', 'Name', 'Sensor','Fecha','Valor'])
Villeta0240

In [ ]:
Villeta0240['Valor'].value_counts()

In [ ]:
Villeta0240['Station'].nunique()

In [ ]:
# Se genera la lista de las diferentes estaciones con base en sus códigos y se muestra cuáles son
j = Villeta0240['Station'].values.ravel()
k = pd.unique(j).tolist()
k

In [ ]:
# Se exportan los datos separados por estaciones según su código para futuros usos 
for i in k:
    Filtro = Villeta0240['Station'] == i
    h = Villeta0240[Filtro]
    nombre_archivo = f"Estacion_{i}.csv"
    print('Se guardó el archivo:', nombre_archivo)
    h.to_csv(nombre_archivo)

In [ ]:
Villeta0240['Valor'].max()

In [ ]:
Villeta0240["Fecha"] = pd.to_datetime(Villeta0240['Fecha'], format = '%Y-%m-%d %H:%M:%S.%f')

In [ ]:
Villeta0240['Fecha'] = Villeta0240['Fecha'] - pd.offsets.DateOffset(hours=7)

In [ ]:
agrupar_dia_0240_03_1_1 = Villeta0240.groupby(by = ["Station", "Name", Villeta0240['Fecha'].dt.date]).agg({'Valor': ['sum','count']})
agrupar_dia_0240_03_1_1

In [ ]:
Filtacum = agrupar_dia_0240_03_1_1[('Valor',   'sum')].max()

In [ ]:
# Se establece el ploteo del acumulado calculado en la celda anterior
fig = plt.figure()
ax = agrupar_dia_0240_03_1_1[('Valor',   'sum')].plot(color='c')
ax.xaxis.set_tick_params(rotation=45)

# Se fijan las etiquetas y título
plt.title('Precipitación diaria EMA', fontsize=14, fontweight="bold")
plt.xlabel('Día')
plt.ylabel('Precipitación (mm/día)')


plt.show()

In [ ]:
dfmax_03_1_1 = agrupar_dia_0240_03_1_1.loc[agrupar_dia_0240_03_1_1[('Valor', 'sum')].idxmax()]
dfmax_03_1_1

In [ ]:
agrupar_dia_0240_03_1_1.to_excel("SumDiaMeteorl2003-1-1.xlsx")

____________

## Extracción de datos de precipitación diezminutal EMA semestral

In [ ]:
def descarga_precip_cassandra(inicio, fin):
    cur.execute(f''' SELECT lm.station, s.name, lm.sensor, lm.event_time + interval '5' hour event_time, lm.event_value 
    FROM cassandra.raw.weather_events AS lm INNER JOIN cassandra.raw.stations AS s
        ON lm.station = s.stationid
    WHERE lm.event_time BETWEEN timestamp '{inicio}' AND timestamp '{fin}' 
    AND lm.sensor in ('0104') ''') # Cambiar código del sensor para cada caso
    station_sensem = cur.fetchall()
    
    # Se genera un Data Frame a partir de los datos extaídos
    stationdf = pd.DataFrame(station_sensem, columns= ['Station', 'Name', 'Sensor','Fecha','Valor'])
    
    # Se genera la lista de las diferentes estaciones con base en sus códigos
    j = stationdf['Station'].values.ravel()
    k = pd.unique(j).tolist()

    # Se crea una carpeta con el nombre del semestre
    inicio_date = datetime.strptime(inicio, '%Y-%m-%d %H:%M:%S.%f')
    fin_date = datetime.strptime(fin, '%Y-%m-%d %H:%M:%S.%f')
    semestre = inicio_date.strftime('%Y-%m') + "_to_" + fin_date.strftime('%Y-%m')
    if not os.path.exists(semestre):
        os.makedirs(semestre)

    # Se exportan los datos separados por estaciones según su código para futuros usos 
    for i in k:
        Filtro = stationdf['Station'] == i
        h = stationdf[Filtro]
        nombre_archivo = semestre + "/Estacion_" + str(i) + ".csv"
        print('Se guardó el archivo:', nombre_archivo)
        h.to_csv(nombre_archivo)
    
    return station_sensem, stationdf

In [ ]:
# Llamar la función y asignar los resultados
EMA_0240_23_2, df_EMA_02a40_23_2 = descarga_precip_cassandra('2023-01-01 00:00:00.000','2024-09-30 23:59:00.000')

In [ ]:
# Llamar la función y asignar los resultados
EMA_0240_23_2, df_EMA_0240_23_2 = descarga_precip_cassandra('2023-01-01 00:00:00.000','2024-09-30 23:59:00.000')

_________

## Extracción de datos masivos por semestre de cualquier sensor

In [ ]:
def descarga_cualqsens_cassandra(inicio, fin, sensor):
    cur.execute(f''' select lm.station, s.name, lm.sensor, lm.event_time + interval '5' hour event_time, lm.event_value 
    from cassandra.raw.weather_events AS lm INNER JOIN cassandra.raw.stations AS s
        ON lm.station = s.stationid
    where lm.event_time BETWEEN timestamp '{inicio}' AND timestamp '{fin}' 
    AND lm.sensor in ('{sensor}') ''') # Cambiar código del sensor para cada caso
    station_sensem = cur.fetchall()
    
    # Se genera un Data Frame a partir de los datos extaídos
    stationdf = pd.DataFrame(station_sensem, columns= ['Station', 'Name', 'Sensor','Fecha','Valor'])
    
    # Se genera la lista de las diferentes estaciones con base en sus códigos
    j = stationdf['Station'].values.ravel()
    k = pd.unique(j).tolist()

    # Se crea una carpeta con el nombre del semestre
    inicio_date = datetime.strptime(inicio, '%Y-%m-%d %H:%M:%S.%f')
    fin_date = datetime.strptime(fin, '%Y-%m-%d %H:%M:%S.%f')
    semestre = inicio_date.strftime('%Y-%m') + "_a_" + fin_date.strftime('%Y-%m')
    if not os.path.exists(semestre):
        os.makedirs(semestre)

    # Se exportan los datos separados por estaciones según su código para futuros usos 
    for i in k:
        Filtro = stationdf['Station'] == i
        h = stationdf[Filtro]
        nombre_archivo = semestre + "/Estacion_" + str(i) + ".csv"
        print('Se guardó el archivo:', nombre_archivo)
        h.to_csv(nombre_archivo)
    
    return station_sensem, stationdf

In [ ]:
# Llamar la función y asignar los resultados
EMA_99_2, df_EMA_99_2 = descarga_cualqsens_cassandra('1999-07-01 00:00:00.000','1999-12-31 23:59:00.000', '0027')

In [ ]:
# Llamar la función y asignar los resultados
EMA_99_2, df_EMA_99_2 = descarga_cualqsens_cassandra('2001-01-01 00:00:00.000','2001-06-30 23:59:00.000', '0027')

In [ ]:
def descarga_cualqsens_cassandra(inicio, fin, sensor):
    cur.execute(f''' select lm.station, s.name, lm.sensor, lm.event_time + interval '5' hour event_time, lm.event_value 
    from cassandra.raw.weather_events AS lm INNER JOIN cassandra.raw.stations AS s
        ON lm.station = s.stationid
    where lm.event_time BETWEEN timestamp '{inicio}' AND timestamp '{fin}' 
    AND lm.sensor in ('{sensor}') ''') # Cambiar código del sensor para cada caso
    station_sensem = cur.fetchall()
    
    # Se genera un Data Frame a partir de los datos extaídos
    stationdf = pd.DataFrame(station_sensem, columns= ['Station', 'Name', 'Sensor','Fecha','Valor'])
    
    # Se genera la lista de las diferentes estaciones con base en sus códigos
    j = stationdf['Station'].values.ravel()
    k = pd.unique(j).tolist()

    # Se crea una carpeta con el nombre del semestre
    inicio_date = datetime.strptime(inicio, '%Y-%m-%d %H:%M:%S.%f')
    fin_date = datetime.strptime(fin, '%Y-%m-%d %H:%M:%S.%f')
    semestre = inicio_date.strftime('%Y-%m') + "_a_" + fin_date.strftime('%Y-%m')
    if not os.path.exists(semestre):
        os.makedirs(semestre)

    # Se exportan los datos separados por estaciones según su código para futuros usos 
    for i in k:
        Filtro = stationdf['Station'] == i
        h = stationdf[Filtro]
        nombre_archivo = semestre + "/Estacion_" + str(i) + ".csv"
        print('Se guardó el archivo:', nombre_archivo)
        h.to_csv(nombre_archivo)
    
    return station_sensem, stationdf

In [7]:
# Llamar la función y asignar los resultados
EMA_99_2, df_EMA_99_2 = descarga_cualqsens_cassandra('1999-01-01 00:00:00.000','2005-12-31 23:59:00.000', '0260')

Se guardó el archivo: 1999-01_a_2005-12/Estacion_0024035390.csv
Se guardó el archivo: 1999-01_a_2005-12/Estacion_0015085050.csv
Se guardó el archivo: 1999-01_a_2005-12/Estacion_0021205970.csv
Se guardó el archivo: 1999-01_a_2005-12/Estacion_0035075080.csv
Se guardó el archivo: 1999-01_a_2005-12/Estacion_0025025280.csv
Se guardó el archivo: 1999-01_a_2005-12/Estacion_0021015070.csv
Se guardó el archivo: 1999-01_a_2005-12/Estacion_0026135300.csv
Se guardó el archivo: 1999-01_a_2005-12/Estacion_0029045150.csv
Se guardó el archivo: 1999-01_a_2005-12/Estacion_0021206930.csv
Se guardó el archivo: 1999-01_a_2005-12/Estacion_0026135290.csv
Se guardó el archivo: 1999-01_a_2005-12/Estacion_0026145090.csv
Se guardó el archivo: 1999-01_a_2005-12/Estacion_0053075020.csv
Se guardó el archivo: 1999-01_a_2005-12/Estacion_0026035100.csv
Se guardó el archivo: 1999-01_a_2005-12/Estacion_0025015020.csv
Se guardó el archivo: 1999-01_a_2005-12/Estacion_0024015110.csv
Se guardó el archivo: 1999-01_a_2005-12/

In [8]:
# Llamar la función y asignar los resultados
EMA_99_2, df_EMA_99_2 = descarga_cualqsens_cassandra('2006-01-01 00:00:00.000','2006-12-31 23:59:00.000', '0260')

Se guardó el archivo: 2006-01_a_2006-12/Estacion_0021206980.csv
Se guardó el archivo: 2006-01_a_2006-12/Estacion_0021015050.csv
Se guardó el archivo: 2006-01_a_2006-12/Estacion_0029045150.csv
Se guardó el archivo: 2006-01_a_2006-12/Estacion_0024035390.csv
Se guardó el archivo: 2006-01_a_2006-12/Estacion_0023195240.csv
Se guardó el archivo: 2006-01_a_2006-12/Estacion_0021195170.csv
Se guardó el archivo: 2006-01_a_2006-12/Estacion_0023105060.csv
Se guardó el archivo: 2006-01_a_2006-12/Estacion_0035075080.csv
Se guardó el archivo: 2006-01_a_2006-12/Estacion_0026225060.csv
Se guardó el archivo: 2006-01_a_2006-12/Estacion_0026105250.csv
Se guardó el archivo: 2006-01_a_2006-12/Estacion_0036015020.csv
Se guardó el archivo: 2006-01_a_2006-12/Estacion_0021015060.csv
Se guardó el archivo: 2006-01_a_2006-12/Estacion_0021115010.csv
Se guardó el archivo: 2006-01_a_2006-12/Estacion_0023085260.csv
Se guardó el archivo: 2006-01_a_2006-12/Estacion_0053075020.csv
Se guardó el archivo: 2006-01_a_2006-12/

In [9]:
# Llamar la función y asignar los resultados
EMA_99_2, df_EMA_99_2 = descarga_cualqsens_cassandra('2007-01-01 00:00:00.000','2007-12-31 23:59:00.000', '0260')

Se guardó el archivo: 2007-01_a_2007-12/Estacion_0052055210.csv
Se guardó el archivo: 2007-01_a_2007-12/Estacion_0021206990.csv
Se guardó el archivo: 2007-01_a_2007-12/Estacion_0021115180.csv
Se guardó el archivo: 2007-01_a_2007-12/Estacion_0024015110.csv
Se guardó el archivo: 2007-01_a_2007-12/Estacion_0026225060.csv
Se guardó el archivo: 2007-01_a_2007-12/Estacion_0051025060.csv
Se guardó el archivo: 2007-01_a_2007-12/Estacion_0024035430.csv
Se guardó el archivo: 2007-01_a_2007-12/Estacion_0021206950.csv
Se guardó el archivo: 2007-01_a_2007-12/Estacion_0021195170.csv
Se guardó el archivo: 2007-01_a_2007-12/Estacion_0035075080.csv
Se guardó el archivo: 2007-01_a_2007-12/Estacion_0011105020.csv
Se guardó el archivo: 2007-01_a_2007-12/Estacion_0026105250.csv
Se guardó el archivo: 2007-01_a_2007-12/Estacion_0029015040.csv
Se guardó el archivo: 2007-01_a_2007-12/Estacion_0021206980.csv
Se guardó el archivo: 2007-01_a_2007-12/Estacion_0021115010.csv
Se guardó el archivo: 2007-01_a_2007-12/

In [10]:
# Llamar la función y asignar los resultados
EMA_99_2, df_EMA_99_2 = descarga_cualqsens_cassandra('2008-01-01 00:00:00.000','2008-12-31 23:59:00.000', '0260')

Se guardó el archivo: 2008-01_a_2008-12/Estacion_0023105060.csv
Se guardó el archivo: 2008-01_a_2008-12/Estacion_0035185010.csv
Se guardó el archivo: 2008-01_a_2008-12/Estacion_0029015040.csv
Se guardó el archivo: 2008-01_a_2008-12/Estacion_0021015070.csv
Se guardó el archivo: 2008-01_a_2008-12/Estacion_0021185090.csv
Se guardó el archivo: 2008-01_a_2008-12/Estacion_0021206990.csv
Se guardó el archivo: 2008-01_a_2008-12/Estacion_0021206950.csv
Se guardó el archivo: 2008-01_a_2008-12/Estacion_0013085050.csv
Se guardó el archivo: 2008-01_a_2008-12/Estacion_0055015010.csv
Se guardó el archivo: 2008-01_a_2008-12/Estacion_0048015040.csv
Se guardó el archivo: 2008-01_a_2008-12/Estacion_0052035040.csv
Se guardó el archivo: 2008-01_a_2008-12/Estacion_0026035100.csv
Se guardó el archivo: 2008-01_a_2008-12/Estacion_0035095120.csv
Se guardó el archivo: 2008-01_a_2008-12/Estacion_0028025120.csv
Se guardó el archivo: 2008-01_a_2008-12/Estacion_0021015050.csv
Se guardó el archivo: 2008-01_a_2008-12/

In [11]:
# Llamar la función y asignar los resultados
EMA_99_2, df_EMA_99_2 = descarga_cualqsens_cassandra('2009-01-01 00:00:00.000','2009-12-31 23:59:00.000', '0260')

Se guardó el archivo: 2009-01_a_2009-12/Estacion_0026035100.csv
Se guardó el archivo: 2009-01_a_2009-12/Estacion_0026145090.csv
Se guardó el archivo: 2009-01_a_2009-12/Estacion_0021015060.csv
Se guardó el archivo: 2009-01_a_2009-12/Estacion_0036015020.csv
Se guardó el archivo: 2009-01_a_2009-12/Estacion_0029045150.csv
Se guardó el archivo: 2009-01_a_2009-12/Estacion_0024015110.csv
Se guardó el archivo: 2009-01_a_2009-12/Estacion_0021205970.csv
Se guardó el archivo: 2009-01_a_2009-12/Estacion_0052035040.csv
Se guardó el archivo: 2009-01_a_2009-12/Estacion_0026095320.csv
Se guardó el archivo: 2009-01_a_2009-12/Estacion_0023105060.csv
Se guardó el archivo: 2009-01_a_2009-12/Estacion_0027015280.csv
Se guardó el archivo: 2009-01_a_2009-12/Estacion_0024055080.csv
Se guardó el archivo: 2009-01_a_2009-12/Estacion_0015085050.csv
Se guardó el archivo: 2009-01_a_2009-12/Estacion_0021206980.csv
Se guardó el archivo: 2009-01_a_2009-12/Estacion_0021206950.csv
Se guardó el archivo: 2009-01_a_2009-12/

In [12]:
# Llamar la función y asignar los resultados
EMA_99_2, df_EMA_99_2 = descarga_cualqsens_cassandra('2010-01-01 00:00:00.000','2010-12-31 23:59:00.000', '0260')

Se guardó el archivo: 2010-01_a_2010-12/Estacion_0026095320.csv
Se guardó el archivo: 2010-01_a_2010-12/Estacion_0037015030.csv
Se guardó el archivo: 2010-01_a_2010-12/Estacion_0052055150.csv
Se guardó el archivo: 2010-01_a_2010-12/Estacion_0023105060.csv
Se guardó el archivo: 2010-01_a_2010-12/Estacion_0055015010.csv
Se guardó el archivo: 2010-01_a_2010-12/Estacion_0048015040.csv
Se guardó el archivo: 2010-01_a_2010-12/Estacion_0021206990.csv
Se guardó el archivo: 2010-01_a_2010-12/Estacion_0025015020.csv
Se guardó el archivo: 2010-01_a_2010-12/Estacion_0053075020.csv
Se guardó el archivo: 2010-01_a_2010-12/Estacion_0052025080.csv
Se guardó el archivo: 2010-01_a_2010-12/Estacion_0021185090.csv
Se guardó el archivo: 2010-01_a_2010-12/Estacion_0021015050.csv
Se guardó el archivo: 2010-01_a_2010-12/Estacion_0021205970.csv
Se guardó el archivo: 2010-01_a_2010-12/Estacion_0024055080.csv
Se guardó el archivo: 2010-01_a_2010-12/Estacion_0026105250.csv
Se guardó el archivo: 2010-01_a_2010-12/

In [13]:
# Llamar la función y asignar los resultados
EMA_99_2, df_EMA_99_2 = descarga_cualqsens_cassandra('2011-01-01 00:00:00.000','2011-12-31 23:59:00.000', '0260')

Se guardó el archivo: 2011-01_a_2011-12/Estacion_0026225060.csv
Se guardó el archivo: 2011-01_a_2011-12/Estacion_0035185010.csv
Se guardó el archivo: 2011-01_a_2011-12/Estacion_0021205970.csv
Se guardó el archivo: 2011-01_a_2011-12/Estacion_0035095120.csv
Se guardó el archivo: 2011-01_a_2011-12/Estacion_0024035430.csv
Se guardó el archivo: 2011-01_a_2011-12/Estacion_0057025020.csv
Se guardó el archivo: 2011-01_a_2011-12/Estacion_0023105060.csv
Se guardó el archivo: 2011-01_a_2011-12/Estacion_0035025110.csv
Se guardó el archivo: 2011-01_a_2011-12/Estacion_0052035040.csv
Se guardó el archivo: 2011-01_a_2011-12/Estacion_0011135030.csv
Se guardó el archivo: 2011-01_a_2011-12/Estacion_0024025090.csv
Se guardó el archivo: 2011-01_a_2011-12/Estacion_0026095320.csv
Se guardó el archivo: 2011-01_a_2011-12/Estacion_0051025060.csv
Se guardó el archivo: 2011-01_a_2011-12/Estacion_0028025120.csv
Se guardó el archivo: 2011-01_a_2011-12/Estacion_0016015120.csv
Se guardó el archivo: 2011-01_a_2011-12/

In [14]:
# Llamar la función y asignar los resultados
EMA_99_2, df_EMA_99_2 = descarga_cualqsens_cassandra('2012-01-01 00:00:00.000','2012-12-31 23:59:00.000', '0260')

Se guardó el archivo: 2012-01_a_2012-12/Estacion_0029065130.csv
Se guardó el archivo: 2012-01_a_2012-12/Estacion_0027015280.csv
Se guardó el archivo: 2012-01_a_2012-12/Estacion_0021195170.csv
Se guardó el archivo: 2012-01_a_2012-12/Estacion_0026095320.csv
Se guardó el archivo: 2012-01_a_2012-12/Estacion_0035025110.csv
Se guardó el archivo: 2012-01_a_2012-12/Estacion_0037015030.csv
Se guardó el archivo: 2012-01_a_2012-12/Estacion_0052055150.csv
Se guardó el archivo: 2012-01_a_2012-12/Estacion_0025015020.csv
Se guardó el archivo: 2012-01_a_2012-12/Estacion_0023105060.csv
Se guardó el archivo: 2012-01_a_2012-12/Estacion_0023085260.csv
Se guardó el archivo: 2012-01_a_2012-12/Estacion_0029015040.csv
Se guardó el archivo: 2012-01_a_2012-12/Estacion_0024025090.csv
Se guardó el archivo: 2012-01_a_2012-12/Estacion_0025025280.csv
Se guardó el archivo: 2012-01_a_2012-12/Estacion_0026225060.csv
Se guardó el archivo: 2012-01_a_2012-12/Estacion_0052055210.csv
Se guardó el archivo: 2012-01_a_2012-12/

In [15]:
# Llamar la función y asignar los resultados
EMA_99_2, df_EMA_99_2 = descarga_cualqsens_cassandra('2013-01-01 00:00:00.000','2013-12-31 23:59:00.000', '0260')

Se guardó el archivo: 2013-01_a_2013-12/Estacion_0048015040.csv
Se guardó el archivo: 2013-01_a_2013-12/Estacion_0026225060.csv
Se guardó el archivo: 2013-01_a_2013-12/Estacion_0026135300.csv
Se guardó el archivo: 2013-01_a_2013-12/Estacion_0026145090.csv
Se guardó el archivo: 2013-01_a_2013-12/Estacion_0024015110.csv
Se guardó el archivo: 2013-01_a_2013-12/Estacion_0035075080.csv
Se guardó el archivo: 2013-01_a_2013-12/Estacion_0021255160.csv
Se guardó el archivo: 2013-01_a_2013-12/Estacion_0026095320.csv
Se guardó el archivo: 2013-01_a_2013-12/Estacion_0015085050.csv
Se guardó el archivo: 2013-01_a_2013-12/Estacion_0027015280.csv
Se guardó el archivo: 2013-01_a_2013-12/Estacion_0024035430.csv
Se guardó el archivo: 2013-01_a_2013-12/Estacion_0029015040.csv
Se guardó el archivo: 2013-01_a_2013-12/Estacion_0021205970.csv
Se guardó el archivo: 2013-01_a_2013-12/Estacion_0029065130.csv
Se guardó el archivo: 2013-01_a_2013-12/Estacion_0021206930.csv
Se guardó el archivo: 2013-01_a_2013-12/

In [16]:
# Llamar la función y asignar los resultados
EMA_99_2, df_EMA_99_2 = descarga_cualqsens_cassandra('2014-01-01 00:00:00.000','2014-12-31 23:59:00.000', '0260')

Se guardó el archivo: 2014-01_a_2014-12/Estacion_0029015040.csv
Se guardó el archivo: 2014-01_a_2014-12/Estacion_0023105060.csv
Se guardó el archivo: 2014-01_a_2014-12/Estacion_0051025060.csv
Se guardó el archivo: 2014-01_a_2014-12/Estacion_0052025080.csv
Se guardó el archivo: 2014-01_a_2014-12/Estacion_0026225060.csv
Se guardó el archivo: 2014-01_a_2014-12/Estacion_0024055070.csv
Se guardó el archivo: 2014-01_a_2014-12/Estacion_0024035430.csv
Se guardó el archivo: 2014-01_a_2014-12/Estacion_0021255160.csv
Se guardó el archivo: 2014-01_a_2014-12/Estacion_0048015040.csv
Se guardó el archivo: 2014-01_a_2014-12/Estacion_0021206950.csv
Se guardó el archivo: 2014-01_a_2014-12/Estacion_0024035390.csv
Se guardó el archivo: 2014-01_a_2014-12/Estacion_0021195190.csv
Se guardó el archivo: 2014-01_a_2014-12/Estacion_0057025020.csv
Se guardó el archivo: 2014-01_a_2014-12/Estacion_0021015050.csv
Se guardó el archivo: 2014-01_a_2014-12/Estacion_0021195170.csv
Se guardó el archivo: 2014-01_a_2014-12/

In [17]:
# Llamar la función y asignar los resultados
EMA_99_2, df_EMA_99_2 = descarga_cualqsens_cassandra('2015-01-01 00:00:00.000','2015-12-31 23:59:00.000', '0260')

Se guardó el archivo: 2015-01_a_2015-12/Estacion_0051025060.csv
Se guardó el archivo: 2015-01_a_2015-12/Estacion_0021015060.csv
Se guardó el archivo: 2015-01_a_2015-12/Estacion_0024025090.csv
Se guardó el archivo: 2015-01_a_2015-12/Estacion_0025025280.csv
Se guardó el archivo: 2015-01_a_2015-12/Estacion_0021206930.csv
Se guardó el archivo: 2015-01_a_2015-12/Estacion_0021195190.csv
Se guardó el archivo: 2015-01_a_2015-12/Estacion_0048015040.csv
Se guardó el archivo: 2015-01_a_2015-12/Estacion_0024035390.csv
Se guardó el archivo: 2015-01_a_2015-12/Estacion_0016015120.csv
Se guardó el archivo: 2015-01_a_2015-12/Estacion_0021255160.csv
Se guardó el archivo: 2015-01_a_2015-12/Estacion_0055015010.csv
Se guardó el archivo: 2015-01_a_2015-12/Estacion_0021206790.csv
Se guardó el archivo: 2015-01_a_2015-12/Estacion_0026105250.csv
Se guardó el archivo: 2015-01_a_2015-12/Estacion_0026095320.csv
Se guardó el archivo: 2015-01_a_2015-12/Estacion_0027015280.csv
Se guardó el archivo: 2015-01_a_2015-12/

In [18]:
# Llamar la función y asignar los resultados
EMA_99_2, df_EMA_99_2 = descarga_cualqsens_cassandra('2016-01-01 00:00:00.000','2016-12-31 23:59:00.000', '0260')

Se guardó el archivo: 2016-01_a_2016-12/Estacion_0025015020.csv
Se guardó el archivo: 2016-01_a_2016-12/Estacion_0021015060.csv
Se guardó el archivo: 2016-01_a_2016-12/Estacion_0026225060.csv
Se guardó el archivo: 2016-01_a_2016-12/Estacion_0027015320.csv
Se guardó el archivo: 2016-01_a_2016-12/Estacion_0012015110.csv
Se guardó el archivo: 2016-01_a_2016-12/Estacion_0035185010.csv
Se guardó el archivo: 2016-01_a_2016-12/Estacion_0027015280.csv
Se guardó el archivo: 2016-01_a_2016-12/Estacion_0024025090.csv
Se guardó el archivo: 2016-01_a_2016-12/Estacion_0025025280.csv
Se guardó el archivo: 2016-01_a_2016-12/Estacion_0016015120.csv
Se guardó el archivo: 2016-01_a_2016-12/Estacion_0023085260.csv
Se guardó el archivo: 2016-01_a_2016-12/Estacion_0021205970.csv
Se guardó el archivo: 2016-01_a_2016-12/Estacion_0025025340.csv
Se guardó el archivo: 2016-01_a_2016-12/Estacion_0015085050.csv
Se guardó el archivo: 2016-01_a_2016-12/Estacion_0048015040.csv
Se guardó el archivo: 2016-01_a_2016-12/

In [19]:
# Llamar la función y asignar los resultados
EMA_99_2, df_EMA_99_2 = descarga_cualqsens_cassandra('2017-01-01 00:00:00.000','2017-12-31 23:59:00.000', '0260')

Se guardó el archivo: 2017-01_a_2017-12/Estacion_0026255030.csv
Se guardó el archivo: 2017-01_a_2017-12/Estacion_0012015110.csv
Se guardó el archivo: 2017-01_a_2017-12/Estacion_0027015280.csv
Se guardó el archivo: 2017-01_a_2017-12/Estacion_0052055210.csv
Se guardó el archivo: 2017-01_a_2017-12/Estacion_0024035430.csv
Se guardó el archivo: 2017-01_a_2017-12/Estacion_0048015040.csv
Se guardó el archivo: 2017-01_a_2017-12/Estacion_0021206930.csv
Se guardó el archivo: 2017-01_a_2017-12/Estacion_0024025090.csv
Se guardó el archivo: 2017-01_a_2017-12/Estacion_0025025280.csv
Se guardó el archivo: 2017-01_a_2017-12/Estacion_0026135300.csv
Se guardó el archivo: 2017-01_a_2017-12/Estacion_0026145090.csv
Se guardó el archivo: 2017-01_a_2017-12/Estacion_0021205970.csv
Se guardó el archivo: 2017-01_a_2017-12/Estacion_0023085260.csv
Se guardó el archivo: 2017-01_a_2017-12/Estacion_0035095120.csv
Se guardó el archivo: 2017-01_a_2017-12/Estacion_0021185090.csv
Se guardó el archivo: 2017-01_a_2017-12/

In [20]:
# Llamar la función y asignar los resultados
EMA_99_2, df_EMA_99_2 = descarga_cualqsens_cassandra('2018-01-01 00:00:00.000','2018-12-31 23:59:00.000', '0260')

Se guardó el archivo: 2018-01_a_2018-12/Estacion_0026225060.csv
Se guardó el archivo: 2018-01_a_2018-12/Estacion_0026145090.csv
Se guardó el archivo: 2018-01_a_2018-12/Estacion_0035025502.csv
Se guardó el archivo: 2018-01_a_2018-12/Estacion_0035025110.csv
Se guardó el archivo: 2018-01_a_2018-12/Estacion_0012015110.csv
Se guardó el archivo: 2018-01_a_2018-12/Estacion_0021205515.csv
Se guardó el archivo: 2018-01_a_2018-12/Estacion_0021015050.csv
Se guardó el archivo: 2018-01_a_2018-12/Estacion_0036015020.csv
Se guardó el archivo: 2018-01_a_2018-12/Estacion_0024055070.csv
Se guardó el archivo: 2018-01_a_2018-12/Estacion_0021205511.csv
Se guardó el archivo: 2018-01_a_2018-12/Estacion_0026135290.csv
Se guardó el archivo: 2018-01_a_2018-12/Estacion_0029015040.csv
Se guardó el archivo: 2018-01_a_2018-12/Estacion_0035095120.csv
Se guardó el archivo: 2018-01_a_2018-12/Estacion_0021205480.csv
Se guardó el archivo: 2018-01_a_2018-12/Estacion_0024055080.csv
Se guardó el archivo: 2018-01_a_2018-12/

In [21]:
# Llamar la función y asignar los resultados
EMA_99_2, df_EMA_99_2 = descarga_cualqsens_cassandra('2019-01-01 00:00:00.000','2019-12-31 23:59:00.000', '0260')

Se guardó el archivo: 2019-01_a_2019-12/Estacion_0024015110.csv
Se guardó el archivo: 2019-01_a_2019-12/Estacion_0021205515.csv
Se guardó el archivo: 2019-01_a_2019-12/Estacion_0026095320.csv
Se guardó el archivo: 2019-01_a_2019-12/Estacion_0021205514.csv
Se guardó el archivo: 2019-01_a_2019-12/Estacion_0012015110.csv
Se guardó el archivo: 2019-01_a_2019-12/Estacion_0021205526.csv
Se guardó el archivo: 2019-01_a_2019-12/Estacion_0024015507.csv
Se guardó el archivo: 2019-01_a_2019-12/Estacion_0023065507.csv
Se guardó el archivo: 2019-01_a_2019-12/Estacion_0021206930.csv
Se guardó el archivo: 2019-01_a_2019-12/Estacion_0023065160.csv
Se guardó el archivo: 2019-01_a_2019-12/Estacion_0021015050.csv
Se guardó el archivo: 2019-01_a_2019-12/Estacion_0021206980.csv
Se guardó el archivo: 2019-01_a_2019-12/Estacion_0021115180.csv
Se guardó el archivo: 2019-01_a_2019-12/Estacion_0035025110.csv
Se guardó el archivo: 2019-01_a_2019-12/Estacion_0048015040.csv
Se guardó el archivo: 2019-01_a_2019-12/

In [22]:
# Llamar la función y asignar los resultados
EMA_99_2, df_EMA_99_2 = descarga_cualqsens_cassandra('2020-01-01 00:00:00.000','2020-12-31 23:59:00.000', '0260')

Se guardó el archivo: 2020-01_a_2020-12/Estacion_0036015020.csv
Se guardó el archivo: 2020-01_a_2020-12/Estacion_0021237060.csv
Se guardó el archivo: 2020-01_a_2020-12/Estacion_0021115180.csv
Se guardó el archivo: 2020-01_a_2020-12/Estacion_0021206980.csv
Se guardó el archivo: 2020-01_a_2020-12/Estacion_0052055090.csv
Se guardó el archivo: 2020-01_a_2020-12/Estacion_0025025340.csv
Se guardó el archivo: 2020-01_a_2020-12/Estacion_0026105250.csv
Se guardó el archivo: 2020-01_a_2020-12/Estacion_0052050080.csv
Se guardó el archivo: 2020-01_a_2020-12/Estacion_0021205514.csv
Se guardó el archivo: 2020-01_a_2020-12/Estacion_0024015507.csv
Se guardó el archivo: 2020-01_a_2020-12/Estacion_0026225060.csv
Se guardó el archivo: 2020-01_a_2020-12/Estacion_0021205513.csv
Se guardó el archivo: 2020-01_a_2020-12/Estacion_0035075070.csv
Se guardó el archivo: 2020-01_a_2020-12/Estacion_0053075020.csv
Se guardó el archivo: 2020-01_a_2020-12/Estacion_0035025110.csv
Se guardó el archivo: 2020-01_a_2020-12/

In [23]:
# Llamar la función y asignar los resultados
EMA_99_2, df_EMA_99_2 = descarga_cualqsens_cassandra('2021-01-01 00:00:00.000','2021-12-31 23:59:00.000', '0260')

Se guardó el archivo: 2021-01_a_2021-12/Estacion_0021237060.csv
Se guardó el archivo: 2021-01_a_2021-12/Estacion_0021205512.csv
Se guardó el archivo: 2021-01_a_2021-12/Estacion_0023195240.csv
Se guardó el archivo: 2021-01_a_2021-12/Estacion_0021205509.csv
Se guardó el archivo: 2021-01_a_2021-12/Estacion_0021205524.csv
Se guardó el archivo: 2021-01_a_2021-12/Estacion_0024015110.csv
Se guardó el archivo: 2021-01_a_2021-12/Estacion_0035260050.csv
Se guardó el archivo: 2021-01_a_2021-12/Estacion_3526500201.csv
Se guardó el archivo: 2021-01_a_2021-12/Estacion_0016015120.csv
Se guardó el archivo: 2021-01_a_2021-12/Estacion_0029015040.csv
Se guardó el archivo: 2021-01_a_2021-12/Estacion_0026225060.csv
Se guardó el archivo: 2021-01_a_2021-12/Estacion_0021205511.csv
Se guardó el archivo: 2021-01_a_2021-12/Estacion_0021205523.csv
Se guardó el archivo: 2021-01_a_2021-12/Estacion_0021205518.csv
Se guardó el archivo: 2021-01_a_2021-12/Estacion_0024035430.csv
Se guardó el archivo: 2021-01_a_2021-12/

In [24]:
# Llamar la función y asignar los resultados
EMA_99_2, df_EMA_99_2 = descarga_cualqsens_cassandra('2022-01-01 00:00:00.000','2022-12-31 23:59:00.000', '0260')

Se guardó el archivo: 2022-01_a_2022-12/Estacion_0037015030.csv
Se guardó el archivo: 2022-01_a_2022-12/Estacion_0052055150.csv
Se guardó el archivo: 2022-01_a_2022-12/Estacion_0021205519.csv
Se guardó el archivo: 2022-01_a_2022-12/Estacion_0052050090.csv
Se guardó el archivo: 2022-01_a_2022-12/Estacion_0021195170.csv
Se guardó el archivo: 2022-01_a_2022-12/Estacion_0025025100.csv
Se guardó el archivo: 2022-01_a_2022-12/Estacion_0026095320.csv
Se guardó el archivo: 2022-01_a_2022-12/Estacion_0021205524.csv
Se guardó el archivo: 2022-01_a_2022-12/Estacion_0029065130.csv
Se guardó el archivo: 2022-01_a_2022-12/Estacion_0021205522.csv
Se guardó el archivo: 2022-01_a_2022-12/Estacion_3526500201.csv
Se guardó el archivo: 2022-01_a_2022-12/Estacion_0035260050.csv
Se guardó el archivo: 2022-01_a_2022-12/Estacion_0021205525.csv
Se guardó el archivo: 2022-01_a_2022-12/Estacion_0035025502.csv
Se guardó el archivo: 2022-01_a_2022-12/Estacion_0035075080.csv
Se guardó el archivo: 2022-01_a_2022-12/

In [25]:
# Llamar la función y asignar los resultados
EMA_99_2, df_EMA_99_2 = descarga_cualqsens_cassandra('2023-01-01 00:00:00.000','2023-12-31 23:59:00.000', '0260')

Se guardó el archivo: 2023-01_a_2023-12/Estacion_0021205512.csv
Se guardó el archivo: 2023-01_a_2023-12/Estacion_0027015280.csv
Se guardó el archivo: 2023-01_a_2023-12/Estacion_0023065507.csv
Se guardó el archivo: 2023-01_a_2023-12/Estacion_0021205518.csv
Se guardó el archivo: 2023-01_a_2023-12/Estacion_0052055220.csv
Se guardó el archivo: 2023-01_a_2023-12/Estacion_0035075090.csv
Se guardó el archivo: 2023-01_a_2023-12/Estacion_0024015508.csv
Se guardó el archivo: 2023-01_a_2023-12/Estacion_0021205517.csv
Se guardó el archivo: 2023-01_a_2023-12/Estacion_0024015507.csv
Se guardó el archivo: 2023-01_a_2023-12/Estacion_0021205525.csv
Se guardó el archivo: 2023-01_a_2023-12/Estacion_0021205513.csv
Se guardó el archivo: 2023-01_a_2023-12/Estacion_0021206930.csv
Se guardó el archivo: 2023-01_a_2023-12/Estacion_0021205519.csv
Se guardó el archivo: 2023-01_a_2023-12/Estacion_0021237060.csv
Se guardó el archivo: 2023-01_a_2023-12/Estacion_0016015120.csv
Se guardó el archivo: 2023-01_a_2023-12/

In [26]:
# Llamar la función y asignar los resultados
EMA_99_2, df_EMA_99_2 = descarga_cualqsens_cassandra('2024-01-01 00:00:00.000','2024-10-14 23:59:00.000', '0260')

Se guardó el archivo: 2024-01_a_2024-10/Estacion_0021205518.csv
Se guardó el archivo: 2024-01_a_2024-10/Estacion_0021205519.csv
Se guardó el archivo: 2024-01_a_2024-10/Estacion_0021205523.csv
Se guardó el archivo: 2024-01_a_2024-10/Estacion_0035075070.csv
Se guardó el archivo: 2024-01_a_2024-10/Estacion_0021205527.csv
Se guardó el archivo: 2024-01_a_2024-10/Estacion_0021205512.csv
Se guardó el archivo: 2024-01_a_2024-10/Estacion_0027015280.csv
Se guardó el archivo: 2024-01_a_2024-10/Estacion_0023065507.csv
Se guardó el archivo: 2024-01_a_2024-10/Estacion_0052050080.csv
Se guardó el archivo: 2024-01_a_2024-10/Estacion_0021205515.csv
Se guardó el archivo: 2024-01_a_2024-10/Estacion_0035075090.csv
Se guardó el archivo: 2024-01_a_2024-10/Estacion_0055015010.csv
Se guardó el archivo: 2024-01_a_2024-10/Estacion_0026225060.csv
Se guardó el archivo: 2024-01_a_2024-10/Estacion_0037015030.csv
Se guardó el archivo: 2024-01_a_2024-10/Estacion_0052050060.csv
Se guardó el archivo: 2024-01_a_2024-10/

____

## Extracción de datos masivos cualquier sensor y estaciones

In [ ]:
temp_codlist = ['003507504', '0014015080', '0013075050', '1303500074',
                '0013065020', '0015065180', '0015045010', '0015015120',
                '0015015050', '0015015100', '1509500118', '0015015060',
                '0025025270', '0028035070', '0028035060', '0028035020',
                '0028035010', '0028035040']

In [ ]:
p_codlist = ['0028035060']

In [ ]:
variables_codlist = ['PTPM_CON']
                     #'0015015100', '0016027310', '0017025020','0011017020']

In [ ]:
def desc_datoscrud_cassandra(inicio, fin, sensor, codestacion):
    cur.execute(f''' select lm.station, s.name, lm.sensor, lm.event_time + interval '5' hour event_time, lm.event_value 
    from cassandra.raw.weather_events AS lm INNER JOIN cassandra.raw.stations AS s
        ON lm.station = s.stationid
    where lm.station in ('{codestacion}')
    AND lm.event_time BETWEEN timestamp '{inicio}' AND timestamp '{fin}' 
    AND lm.sensor in ('{sensor}') ''') # estación de la forma '0021206600'; sensor de la forma '0240'
    station_sensem = cur.fetchall()
    
    # Se genera un Data Frame a partir de los datos extaídos
    stationdf = pd.DataFrame(station_sensem, columns= ['Station', 'Name', 'Sensor','Fecha','Valor'])
    
    # Se genera la lista de las diferentes estaciones con base en sus códigos
    j = stationdf['Station'].values.ravel()
    k = pd.unique(j).tolist()

    # Se crea una carpeta con el nombre del semestre
    inicio_date = datetime.strptime(inicio, '%Y-%m-%d %H:%M:%S.%f')
    fin_date = datetime.strptime(fin, '%Y-%m-%d %H:%M:%S.%f')
    semestre = inicio_date.strftime('%Y-%m') + "_a_" + fin_date.strftime('%Y-%m')
    if not os.path.exists(semestre):
        os.makedirs(semestre)

    # Se exportan los datos separados por estaciones según su código para futuros usos 
    for i in k:
        Filtro = stationdf['Station'] == i
        h = stationdf[Filtro]
        nombre_archivo = semestre + "/Estacion_" + str(i) + ".csv"
        print('Se guardó el archivo:', nombre_archivo)
        h.to_csv(nombre_archivo)
    
    return station_sensem, stationdf

In [ ]:
# Llamar la función y asignar los resultados
for i in p_codlist:
    EMA_0103_23_2, df_EMA_0103_23_2 = desc_datoscrud_cassandra('2023-03-16 00:00:00.000','2024-09-30 23:59:00.000',
                                                               '0068', i)
    print(f'Analizada estación {i}')

In [ ]:
# Llamar la función y asignar los resultados
EMA_99_2, df_EMA_99_2 = descarga_cualqsens_cassandra('2021-12-31 00:00:00.000','2021-07-15 23:59:00.000', '0027')